## Introduction

The COVID-19 pandemic has impacted much of the global supply chain. In particular, we wish to investigate its impact on the US’s agricultural truck transportation supply chain and the price/volume of certain agricultural products. For example, we want to investigate the correlation between COVID-19 cases, truck volume, and agricultural product pricing in the state of California and Florida.

This notebook details our process for importing data sets, cleaning data sets, merging data sets, creating visualizations, and conducting analysis.

First, we must install `openpyxl` if not already. We follow Jake Vanderplas's [advice](https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/) on installing packages within notebooks.

In [1]:
import sys
!{sys.executable} -m pip install openpyxl

     |████████████████████████████████| 256kB 35.9MB/s eta 0:00:01


In [2]:
pip install altair vega_datasets

     |████████████████████████████████| 819kB 43.5MB/s eta 0:00:01
     |████████████████████████████████| 215kB 79.6MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [29]:
# import data science packages
import numpy as np
import pandas as pd
import re
# import visualization related packages
import matplotlib.pyplot as plt
import numpy as np
import altair as alt

## Truck Data

TODO: Give brief introduction for the truck data and where/how we got the data.

In [34]:
# import the truck data

START = 2015 # The starting year to begin loading the truck data
END = 2022   # The ending year (inclusive) to stop loading the truck data.
TRACE = True # Boolean value whether to trace the output.

lst = []
for year in range(START, END+1):
    filename = f'Refrigerated_Truck_Volumes_{year}.csv'
    temp_df = pd.read_csv(filename)
    lst.append(temp_df)

    if TRACE:
        print(f"The file '{filename}' contains {temp_df.shape[0]} rows and {temp_df.shape[1]} columns.")

# concatenate all data frames together
truck_df = pd.concat(lst, axis=0, ignore_index=True)

if TRACE:
    print(f"The full data frame contains {truck_df.shape[0]} rows and {truck_df.shape[1]} columns.")

The file 'Refrigerated_Truck_Volumes_2015.csv' contains 114836 rows and 14 columns.
The file 'Refrigerated_Truck_Volumes_2016.csv' contains 115813 rows and 14 columns.
The file 'Refrigerated_Truck_Volumes_2017.csv' contains 115112 rows and 14 columns.
The file 'Refrigerated_Truck_Volumes_2018.csv' contains 116071 rows and 14 columns.
The file 'Refrigerated_Truck_Volumes_2019.csv' contains 121176 rows and 14 columns.
The file 'Refrigerated_Truck_Volumes_2020.csv' contains 129428 rows and 14 columns.
The file 'Refrigerated_Truck_Volumes_2021.csv' contains 140029 rows and 14 columns.
The file 'Refrigerated_Truck_Volumes_2022.csv' contains 137708 rows and 14 columns.
The full data frame contains 990173 rows and 14 columns.


In [35]:
# Some rows for truck volume may be assigned to different commodity marketing 
#   seasons (e.g., a shipment on 1/1/2015 may be assigned to 2014, 2015, or both)
#   indicated by the `Season` column. If both, then there will be 2 rows of data.
# For our analysis, we disregard this fact because we are more interested in the 
#   `date` of the shipment.
# Therefore, we first group the data, then aggregate by summation of the `10,000 LBS` 
#   column and finally we drop the `Season` column.
groupby_cols = truck_df.columns.drop(['Season','10,000 LBS']).tolist()
clean_truck_df = truck_df.groupby(by=groupby_cols)[['10,000 LBS']].sum().reset_index()

In [36]:
clean_truck_df.head()

,date,Weekday,Month,Year,Quarter,Tuesday Week Ending,Tuesday Week Ending Number,Mode,Region,Origin,District,Commodity,"10,000 LBS"
0,01/01/2015,4,1,2015,1,01/06/2015,1,Truck,Arizona,Arizona,Western Arizona,"Lettuce, Green Leaf",0
1,01/01/2015,4,1,2015,1,01/06/2015,1,Truck,Arizona,Arizona,Western Arizona,"Lettuce, Iceberg",19
2,01/01/2015,4,1,2015,1,01/06/2015,1,Truck,Arizona,Arizona,Western Arizona,"Lettuce, Romaine",3
3,01/01/2015,4,1,2015,1,01/06/2015,1,Truck,Florida,Florida,Florida Districts,Beans,15
4,01/01/2015,4,1,2015,1,01/06/2015,1,Truck,Florida,Florida,Florida Districts,"Corn, Sweet",14


In [37]:
# The column descriptions are taken from the data set page:
# https://agtransport.usda.gov/Truck/Refrigerated-Truck-Volumes/rfpn-7etz
cols_to_keep = [
    'date',      # reporting date
    'Month',     
    'Year',      
    'Mode',      # all origins are domestic, mode is either truck or import, both of
                 #   which are truck movements
    'Region',    # broader region assigned to `Origin` by USDA's Transporation
                 #   Services Division 
    'Origin',    # broader region assigned to `District` by Market News Speciualty Crops
    'District',  # Origin Specialty Crops district
    'Commodity', # commodity (either fruit or vegetable)
    '10,000 LBS' # integer value for truck volume in 10,000 lbs
]
clean_truck_df = clean_truck_df[cols_to_keep]

In [38]:
clean_truck_df

,date,Month,Year,Mode,Region,Origin,District,Commodity,"10,000 LBS"
0,01/01/2015,1,2015,Truck,Arizona,Arizona,Western Arizona,"Lettuce, Green Leaf",0
1,01/01/2015,1,2015,Truck,Arizona,Arizona,Western Arizona,"Lettuce, Iceberg",19
2,01/01/2015,1,2015,Truck,Arizona,Arizona,Western Arizona,"Lettuce, Romaine",3
3,01/01/2015,1,2015,Truck,Florida,Florida,Florida Districts,Beans,15
4,01/01/2015,1,2015,Truck,Florida,Florida,Florida Districts,"Corn, Sweet",14
5,01/01/2015,1,2015,Truck,Florida,Florida,Florida Districts,Eggplant,7
6,01/01/2015,1,2015,Truck,Florida,Florida,Florida Districts,Squash,9
7,01/01/2015,1,2015,Truck,Florida,Florida,Florida Districts,Strawberries,26
8,01/01/2015,1,2015,Truck,Florida,Florida,Florida Districts,Tomatoes,44
9,01/01/2015,1,2015,Truck,Florida,Florida,Florida Districts,"Tomatoes, Plum Type",5


In [39]:
# Note that the `10,000 LBS` column is encoded as an integer value. We can infer that 
#   truck movements with volumes below the 10,000 lbs threshold will have a value of 0.
#   Since, this is weekly aggregated truck volumes there isn't use in keeping rows
#   with 0 values for the `10,000 LBS` column.
clean_truck_df = clean_truck_df[clean_truck_df['10,000 LBS'] != 0]

if TRACE:
    print(f"The cleaned data frame contains {clean_truck_df.shape[0]} rows and {clean_truck_df.shape[1]} columns.")

The cleaned data frame contains 856759 rows and 9 columns.


In [40]:
# We want to fix the `date` column to be a datetime type.
clean_truck_df.loc[:,'date'] = pd.to_datetime(clean_truck_df['date'], format='%m/%d/%Y')

In [41]:
# check the number of unique values for each column
for col in clean_truck_df.columns:
    print(f"The column `{col}` has {len(clean_truck_df[col].unique())} unique values.")

The column `date` has 2922 unique values.
The column `Month` has 12 unique values.
The column `Year` has 8 unique values.
The column `Mode` has 2 unique values.
The column `Region` has 17 unique values.
The column `Origin` has 42 unique values.
The column `District` has 135 unique values.
The column `Commodity` has 133 unique values.
The column `10,000 LBS` has 2555 unique values.


In [42]:
# For our analysis, we filter the data frame to only include data for California and Florida.
REGIONS = ['California', 'Florida']
filtered_truck_df = clean_truck_df[clean_truck_df['Region'].isin(REGIONS).values]

In [43]:
filtered_truck_df.head(10)

,date,Month,Year,Mode,Region,Origin,District,Commodity,"10,000 LBS"
3,2015-01-01,1,2015,Truck,Florida,Florida,Florida Districts,Beans,15
4,2015-01-01,1,2015,Truck,Florida,Florida,Florida Districts,"Corn, Sweet",14
5,2015-01-01,1,2015,Truck,Florida,Florida,Florida Districts,Eggplant,7
6,2015-01-01,1,2015,Truck,Florida,Florida,Florida Districts,Squash,9
7,2015-01-01,1,2015,Truck,Florida,Florida,Florida Districts,Strawberries,26
8,2015-01-01,1,2015,Truck,Florida,Florida,Florida Districts,Tomatoes,44
9,2015-01-01,1,2015,Truck,Florida,Florida,Florida Districts,"Tomatoes, Plum Type",5
28,2016-01-01,1,2016,Truck,Florida,Florida,Florida Districts,Beans,24
29,2016-01-01,1,2016,Truck,Florida,Florida,Florida Districts,Cabbage,157
30,2016-01-01,1,2016,Truck,Florida,Florida,Florida Districts,"Corn, Sweet",10


## Fruit Price Data

TODO: Give brief introduction for the truck data and where/how we got the data.

In [44]:
# get all sheets names of excel file
f = pd.ExcelFile('selected-weekly-fruit-movement-and-price.xlsx')
sheets = f.sheet_names

In [45]:
# get list of all commodities
fruit_weekly_price = pd.read_excel('selected-weekly-fruit-movement-and-price.xlsx',sheet_name=sheets[0])
fruit_weekly_price = fruit_weekly_price.iloc[6:28,0]

In [46]:
# get price of each week from each sheet
for sheet in sheets:
    week = pd.read_excel('selected-weekly-fruit-movement-and-price.xlsx',sheet_name=sheet)
    week = week.iloc[6:28,17]
    week = pd.to_numeric(week, errors='coerce')
    fruit_weekly_price = pd.concat([fruit_weekly_price, week], axis=1)

In [47]:
# reassign proper column names
col = ['Commodity']

for sheet in sheets:
    dt = sheet[-6:]
    p_date = '20'+dt[-2:]+'-'+dt[:2]+'-'+dt[2:4]
    col.append(p_date)

fruit_weekly_price.columns = col

In [15]:
fruit_weekly_price.head()

,Commodity,2022-04-23,2022-04-16,2022-04-09,2022-04-02,2022-03-26,2022-03-19,2022-03-12,2022-03-05,2022-02-26,...,2020-07-04,2020-06-27,2020-06-20,2020-06-13,2020-06-06,2020-05-30,2020-05-23,2020-05-16,2020-05-09,2020-05-02
6,Apples,1.437777,1.894444,1.623,1.676667,1.575,1.491,1.676,1.523,1.567778,...,1.36875,1.374444,1.228,1.354,1.45625,1.347778,1.198889,1.242222,1.246,1.45
7,Avocados 4/,1.580000,1.510000,1.240,1.570000,1.160,1.370,1.410,1.270,1.240000,...,1.31000,1.080000,1.120,1.270,1.26000,1.110000,1.260000,1.290000,1.240,1.04
8,Bananas,0.520000,0.510000,0.520,0.570000,0.530,0.550,0.380,0.580,0.530000,...,0.44000,0.420000,0.340,0.550,0.37000,0.430000,0.550000,0.510000,0.460,0.51
9,Blueberries,3.020000,3.600000,2.890,3.610000,3.310,2.780,3.250,3.220,2.600000,...,2.59000,2.700000,2.400,2.520,2.90000,2.980000,2.780000,2.590000,2.660,2.77
10,Cantaloupes,0.810000,1.160000,0.570,0.980000,1.870,0.630,0.570,0.860,0.530000,...,0.44000,0.480000,0.460,0.480,0.56000,0.630000,0.570000,0.520000,0.490,0.49


In [71]:
def only_word(commodity):
    return re.sub('[^a-zA-Z]+', '', commodity)

In [72]:
#change to long format
fruit_weekly_price_long = pd.melt(fruit_weekly_price, id_vars = 'Commodity', 
                                  value_vars = sorted(fruit_weekly_price.columns[1:]), 
                                  var_name='date', value_name='price')

#remvoe digit, only keep characters to find common commodities with the commodity column in truck dataset
fruit_weekly_price_long['Commodity'] = fruit_weekly_price_long['Commodity'].apply(lambda x: only_word(x))



In [92]:
#define a function to find common values base on the column given as an input 
def find_common_items(df1, df2, col): 
    df_1_col = df1[col].unique()
    common_commodities =[]
    for i in df2[col]:
        if i in df_1_col:
            common_commodities.append(i)
    return common_commodities
               

In [93]:
common_commodities = find_common_items(filtered_truck_df, fruit_weekly_price, 'Commodity')

In [94]:
#filter common commodity in fruit data set 
fruit_weekly_price_long = fruit_weekly_price_long[fruit_weekly_price_long['Commodity'].isin(common_commodities)]


In [97]:
# fill the nan values with the commodity's previous value (nearest)
bfill_imputation_fruit = fruit_weekly_price_long.groupby('Commodity').fillna(method='bfill')
bfill_imputation_fruit['Commodity'] = fruit_weekly_price_long['Commodity']
bfill_imputation_fruit.head()

,date,price,Commodity
0,2020-05-02,1.450,Apples
3,2020-05-02,2.770,Blueberries
5,2020-05-02,2.990,Cherries
6,2020-05-02,0.440,Grapefruit
7,2020-05-02,2.025,Grapes


In [108]:
#filter common_commodities in truck dataset
filtered_truck_common = filtered_truck_df[filtered_truck_df['Commodity'].isin(common_commodities)]
filtered_truck_common.head()

,date,Month,Year,Mode,Region,Origin,District,Commodity,"10,000 LBS"
7,2015-01-01,1,2015,Truck,Florida,Florida,Florida Districts,Strawberries,26
43,2016-01-01,1,2016,Truck,Florida,Florida,Florida Districts,Strawberries,51
50,2017-01-01,1,2017,Truck,Florida,Florida,Florida Districts,Strawberries,17
55,2018-01-01,1,2018,Truck,California,California-south,Oxnard District,Strawberries,18
64,2018-01-01,1,2018,Truck,Florida,Florida,Florida Districts,Grapefruit,490


In [109]:
# build dataframe for plotting truck volume line 
#group by time, commodity type and region 
group_ser = filtered_truck_common.groupby(['Year', 'Month', 'Commodity', 'Region']).sum()


truck_sum_month = group_ser.reset_index()
truck_sum_month["date"] = truck_sum_month['Year'].astype(str) +"-"+ truck_sum_month["Month"].astype(str)

In [110]:
truck_sum_month.head()

,Year,Month,Commodity,Region,"10,000 LBS",date
0,2015,1,Grapefruit,Florida,8116,2015-1
1,2015,1,Grapes,California,406,2015-1
2,2015,1,Oranges,Florida,3294,2015-1
3,2015,1,Raspberries,California,530,2015-1
4,2015,1,Strawberries,California,2567,2015-1


## COVID-19 Data

TODO: Give brief introduction for the truck data and where/how we got the data.

In [111]:
import pandas as pd 
import altair as alt

In [162]:
data_covid = pd.read_csv ('Weekly_United_States_COVID-19_Cases_and_Deaths_by_State.csv')
#take a look at the shape
data_covid.shape

(9360, 10)

In [163]:
covid_filtered = data_covid[data_covid['state'].isin(['CA', 'FL'])][['date_updated', 'state',  'tot_cases']]
covid_filtered = covid_filtered.rename(columns={"date_updated": "date"})

In [114]:
# define a function to calculate the top n fast growth period based on the total cases 
def find_date(df, n):
    diff_list = []
    for i in range(0, len(df['tot_cases'])-1):
        difference = (df['tot_cases'].iloc[i+1] - df['tot_cases'].iloc[i])
        diff_list.append((difference,df['date_updated'].iloc[i+1]))
        result = sorted(diff_list, reverse = True)
    return result[:n]

In [164]:
find_date(data_covid[data_covid['state'] =='CA'], 2)

[(790954, '2022-01-13'), (788494, '2022-01-20')]

# Code for Visualization 

## visualization code for Covid-19 total cases change with time 

In [160]:
# plot the base line chart of covid-19
def covid_area_plot(df): 
    
    area_covid = alt.Chart(df).mark_area().encode(
        alt.X('date:T', title = 'Date',
              axis=alt.Axis(tickMinStep = 4, labelAngle=45), 
        ),
        alt.Y('tot_cases:Q', 
              title = 'Total Cases Change') , 
              color = 'state:N').properties(
                          title = 'Covid_19 Total Cases Change', 
                          width = 700, 
                          height = 240)


# add annotation of arrow and essential text explanation on the line chart 

    arrow_covid = alt.Chart(pd.DataFrame({'x': ['2021-01-01','2022-01-01'], 
                                          'y': [5500000,12000000], 
                                          'textof': ['➟', '➟']})).mark_text(dx=-25,
                                                                            dy=19, 
                                                                            angle=0, 
                                                                            fontSize=35).encode(
                                            x='x:T',
                                            y='y',
                                            text='textof')
                                            

    text_covid = alt.Chart(pd.DataFrame({'x': ['2021-01-01', '2022-01-01'] , 
                                         'y': [6000000,  13000000], 
                                          'textof': ['First Rapid Growth', 'Second Rapid Growth']})).mark_text(
                                            angle=0, 
                                            color = 'black', 
                                            fontSize=14, 
                                            fontWeight=200, 
                                            opacity=1
                                         ).encode(
                                            x='x:T', y='y:Q', text='textof')
                                     


# concatenate base line chart with text and arrow 
    chart_covid = area_covid + arrow_covid  + text_covid
    return chart_covid

In [161]:
covid_area_plot(covid_filtered)

alt.LayerChart(...)

## visualization code for fruit prices change with time 

In [165]:
# fruit line and interactivity 
def fruit_line_plot(df): 
    metricOptions = common_commodities
    input_dropdown = alt.binding_select(options = metricOptions, name = 'Fruit Price : Commodity')
    dropdown_selection = alt.selection_single(fields = ['Commodity'], bind = input_dropdown)

# base line chart of fruit prices with time
    line_fruit = alt.Chart(df).mark_line().encode(
                alt.X('date:T',
                axis=alt.Axis(tickMinStep = 2, labelAngle=45)),
                y = 'price:Q', 
                color = 'Commodity:N').properties(title = 'Fruit Prices Change', 
                                                  width = 700, 
                                                  height = 240).add_selection(dropdown_selection).transform_filter(dropdown_selection)

# use dotted line to mark the time division before and after 2 covid cases rapid growth periods.
    rules_fruit = alt.Chart(pd.DataFrame({
              'Date': ['2021-01-01', '2022-01-01'],
              'color': ['red', 'red']
                })).mark_rule(size = 3, 
                              color = 'red',
                              strokeDash=[3, 2],
                 ).encode(
                    x='Date:T' )

# combine the base line and annotation together 
    chart_fruit = line_fruit + rules_fruit 
    return chart_fruit

In [166]:
fruit_line_plot(bfill_imputation_fruit)

alt.LayerChart(...)

## visualization code for truck volume change with time 

In [167]:
def truck_line_interactive(df): 
    
    metricOptions = common_commodities
    input_dropdown = alt.binding_select(options = metricOptions, name = 'Truck: Commodity ')
    dropdown_selection = alt.selection_single(fields = ['Commodity'], bind = input_dropdown)


    line_truck_volume = alt.Chart(df).mark_line().encode(
        alt.X('date:T',
              title = 'Date',
              axis=alt.Axis(tickMinStep = 2, labelAngle=45)),
                y = '10,000 LBS:Q', 
                color = 'Region:N').properties(title = 'Truck volume Change', 
                                       width = 700, 
                                      height = 240).add_selection(dropdown_selection).transform_filter(dropdown_selection)

# Add annotation: 
#add a verticle mark line to distinguish the period before and post Covid_19 as the earliest date as 2020-01-23 


    rules_truck = alt.Chart(pd.DataFrame({
      'Date': ['2020-01-23'],
      'color': ['red']
        })).mark_rule(size = 3, 
              color = 'red',
              strokeDash=[3, 2],
             ).encode(
          x='Date:T' )




    text_truck = alt.Chart({'values':[{'x': '2020-07-30', 'y': 72000}]}).mark_text(
        text='2020 Covid_19 Records Started', angle=0, color = 'red'
    ).encode(
        x='x:T', y='y:Q'
    )

    chart_truck = line_truck_volume + rules_truck + text_truck
    return chart_truck

In [169]:
truck_line_interactive(truck_sum_month)

alt.LayerChart(...)